In [ ]:
#pip install -r requirements.txt

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv('creds.env')

True

# Rentalman Connector

In [ ]:
import pyodbc
rm_conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', system=os.getenv('RENTALMAN_URL'),SIGNON=4)
rm_c1 = rm_conn.cursor()

# Pull Data
rm_query = '''
SELECT *
FROM SCHEMA.TABLE
LIMIT 1
'''

rm_c1.execute(rm_query)
# /Pull Data
results = rm_c1.fetchall()

rentalman_df = pd.DataFrame.from_records(results, columns=[x[0] for x in rm_c1.description])

# Converts your dataframe into tuple form for loading into teradata
data = [tuple(row) for row in rentalman_df.values]

# Teradata Connector

In [ ]:
import teradatasql
conn =  teradatasql.connect(host=os.getenv('TERADATA_URL'), user=os.getenv('TERADATA_USERNAME'), password=os.getenv('TERADATA_PASSWORD'))
cursor = conn.cursor()
# Drop Data
drop_table_query = """
DELETE FROM SCHEMA.TABLE;
"""
cursor.execute(drop_table_query)
# /Drop Data

# Loading Data
insert_query = '''
INSERT INTO SCHEMA.TABLE (
COLUMN_FIELD1,
COLUMN_FIELD2

)  VALUES(
?,
?
)
'''
cursor.executemany(insert_query, data)
# / Loading Data


# Pull Data
query = """
Select top 1 * from SCHEMA.TABLE
"""
df=pd.read_sql(query,conn)
# / Pull Data


conn.commit()
cursor.close()
conn.close()

# Snowflake Connector

In [ ]:
from snowflake.snowpark import Session

connection_parameters = {
    "account": os.getenv('SNOWFLAKE_ACCOUNT'),
    "user": os.getenv('SNOWFLAKE_USERNAME'),
    "password": os.getenv('SNOWFLAKE_PASSWORD'),
    "role": os.getenv('ROLE'),
    "warehouse": os.getenv('WAREHOUSE'),
    # OPTIONAL "database": os.getenv('DATABASE'),
    # OPTIONAL "schema": os.getenv('SCHEMA')
    }


session = Session.builder.configs(connection_parameters).create()

#Pull Data
snowflake_df = session.sql("""
                           select top 5 *
from database.schema.table
                           """)
#/Pull Data

# Display data (Not in a pandas dataframe)
#snowflake_df.show()

# Load Query into Pandas
df = pd.DataFrame(snowflake_df.collect())



#Load Pandas into Snowflake
df.write.mode("overwrite").save_as_table("database.schema.table")